# Установки и импорты

In [1]:
!pip install transformers
!pip install pytube
!pip install youtube-transcript-api
!pip install pydub
!apt install ffmpeg
!pip install fairseq2
!pip install pydub sentencepiece
!pip install git+https://github.com/facebookresearch/seamless_communication.git
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.8/191.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import string

import pandas as pd
from jiwer import wer
import torch

from asr_pipeline import YoutubeASR

# Работа с видео

In [3]:
yb_link = 'https://www.youtube.com/watch?v=B6kD0Tzhr8I'
timecode_start='5:04'
timecode_end='5:50'

In [4]:
сaptions = YoutubeASR.get_generated_captions(yb_link, 'ko')
captions = YoutubeASR.crop_captions(сaptions, timecode_start, timecode_end)

In [5]:
mannual_captions = ('지금까지 얘기한 방 외에 어떤 방이 또 있을까요? 보드 게임방이나 방탈출 카페 같은 것도 있다고 들었어요. '
                    '방탈출 카페는 제한 시간 안에 문제를 풀어서 방을 탈출하는 게임을 하는 곳인가요? 맞아요. '
                    '방탈출 게임이 인기가 많아지자 그것을 소재로 한 텔레비전 프로그램들도 나오더라고요. 아, 저도 본 것 같아요. '
                    '강호동 씨가 나오는 대탈출이라는 예능요. 저도 잠깐 본 적이 있는데, 많은 분들이 좋아하더라고요. '
                    '그러니까요. 아무튼 한국에서는 이런 방 문화가 많이 발달한 게 독특해요')

In [6]:
recognized = pd.DataFrame(
    [('youtube', captions), ('mannual', mannual_captions)],
    columns = ['system', 'recognized']
)
recognized

,system,recognized
0,yotube,지금까지 얘기한 방 외에 어떤 방이 또 있을까요 보드게임방 이나 방탈출 카페 같은 ...
1,mannual,지금까지 얘기한 방 외에 어떤 방이 또 있을까요? 보드 게임방이나 방탈출 카페 같은...


# ASR

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [10]:
yb_asr = YoutubeASR()
yb_asr.models_params

{'whisper': {'model_name': 'openai/whisper-tiny'},
 'm4t': {'model_name': 'seamlessM4T_medium', 'vocoder': 'vocoder_36langs'},
 'wav2vec': {'model_name': 'anantoj/wav2vec2-xls-r-1b-korean'}}

## Whisper

Реализовано при помощи библиотеки transformers

In [11]:
whisper_res = yb_asr.recognize(yb_video=yb_link,
                               asr_model='whisper',
                               lang='ko',
                               device=device,
                               start=timecode_start,
                               end=timecode_end)

Trimming audio to B6kD0Tzhr8I_trimmed.mp3


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

In [12]:
whisper_res

' 지금까지 얘기한 방외의 어떤 방이 또 있을까요? 모드게임 방이나 방탈출 카페 같은 것도 있다고 들었어요.  방탈출 카페는 대한 시간 안에 문제를 풀어서 방을 탈출하는 게임을 하는 곳인가요? 맞아요 방탈출 게임이 인기가 많아지죠 그것을 소재로한 텔레비전 프로그램들도 나오더라고요 아 저도 본 것 같아요 강호동 씨가 나오는 대탈출이라는 예능요 저도 잠깐 본 적이 있는데 많은 분들이 좋아하더라고요  그러니까요 아무튼 한국에서는 이런 방문화가 많이 발달한 게 독특해요'

In [13]:
recognized.loc[2] = ('whisper', whisper_res)
recognized

,system,recognized
0,yotube,지금까지 얘기한 방 외에 어떤 방이 또 있을까요 보드게임방 이나 방탈출 카페 같은 ...
1,mannual,지금까지 얘기한 방 외에 어떤 방이 또 있을까요? 보드 게임방이나 방탈출 카페 같은...
2,whisper,지금까지 얘기한 방외의 어떤 방이 또 있을까요? 모드게임 방이나 방탈출 카페 같은...


In [14]:
trimmed_video = 'B6kD0Tzhr8I_trimmed.mp3'

## M4T
Реализовано при помощи библиотеки от фейсбука [seamless_communication](https://github.com/facebookresearch/seamless_communication/tree/main)

In [15]:
m4t_res = yb_asr.recognize(yb_video=trimmed_video,
                           asr_model='m4t',
                           lang='kor',
                           device=device)
m4t_res

100%|██████████| 6.37G/6.37G [07:16<00:00, 15.7MB/s]
100%|██████████| 4.63M/4.63M [00:00<00:00, 169MB/s]
100%|██████████| 160M/160M [00:12<00:00, 13.6MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


'지금까지 얘기한 방 외에 어떤 방이 또 있을까요? 보드게임방이나 방탈출 카페 같은 것도 있다고 들었어요. 방탈출 카페는 제한시간 안에 문제를 풀어서 방을 탈출하는 게임을 하는 곳인가요? 맞아요. 방탈출 게임이 인기가 많아지자 그것을 소재로 한 텔레비전 프로그램들도 나오더라고요. 아, 저도 본 것 같아요. 강호동 씨가 나오는 대탈출이라는 예능이요. 저도 잠깐 봤는데 많은 분들이 좋아더라고요. 그러니까 아무튼 한국에서는 이런 방문화가 많이 발달한 게 독특해요'

In [16]:
recognized.loc[3] = ('m4t', m4t_res)
recognized

,system,recognized
0,yotube,지금까지 얘기한 방 외에 어떤 방이 또 있을까요 보드게임방 이나 방탈출 카페 같은 ...
1,mannual,지금까지 얘기한 방 외에 어떤 방이 또 있을까요? 보드 게임방이나 방탈출 카페 같은...
2,whisper,지금까지 얘기한 방외의 어떤 방이 또 있을까요? 모드게임 방이나 방탈출 카페 같은...
3,m4t,지금까지 얘기한 방 외에 어떤 방이 또 있을까요? 보드게임방이나 방탈출 카페 같은 ...


## Wav2vec

In [17]:
wav2vec_res = yb_asr.recognize(
    yb_video=trimmed_video,
    asr_model='wav2vec',
    lang='ko'
)
wav2vec_res

config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Some weights of the model checkpoint at anantoj/wav2vec2-xls-r-1b-korean were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at anantoj/wav2vec2-xls-r-1b-korean and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN t

tokenizer_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

{'text': '지금까지 얘기한 방외에 어떤 방이 또 있을까요 모득게인방이나 방달출카에 같은 것도 있다고 들었어요탈출카페는 대한 시간 아에 문제를 뿌어서 방을 탈출하는 게임을 하는 것인가요 마자달 게임미 인기가 많아지자 그것을 소에로한 테레기존 프로그램도 나오더라고요  저도 봉 것 같아 강호동시가 나오는 털출이라는 얘는면 저도 장권 본적이 있는데 많은 분들이 좋하더라고그러니까여아파트는 한국에서는 이런 방문화가 많이 발달는 게 도특돼요'}

In [18]:
recognized.loc[4] = ('wav2vec', wav2vec_res['text'])
recognized

,system,recognized
0,yotube,지금까지 얘기한 방 외에 어떤 방이 또 있을까요 보드게임방 이나 방탈출 카페 같은 ...
1,mannual,지금까지 얘기한 방 외에 어떤 방이 또 있을까요? 보드 게임방이나 방탈출 카페 같은...
2,whisper,지금까지 얘기한 방외의 어떤 방이 또 있을까요? 모드게임 방이나 방탈출 카페 같은...
3,m4t,지금까지 얘기한 방 외에 어떤 방이 또 있을까요? 보드게임방이나 방탈출 카페 같은 ...
4,wav2vec,지금까지 얘기한 방외에 어떤 방이 또 있을까요 모득게인방이나 방달출카에 같은 것도 ...


In [19]:
recognized.to_csv('recognized.tsv', sep='\t', index=False, header=True)

# Оценка качества

Перед тем как делать wer уберу токенизацию

In [20]:
def remove_punctuation(row):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in row.recognized if ch not in exclude)

In [21]:
recognized['no_punct'] = recognized.apply(remove_punctuation, axis=1)

In [22]:
recognized['wer'] = recognized.apply(lambda x: wer(recognized.loc[1, 'no_punct'], x.no_punct), axis=1)

In [23]:
recognized

,system,recognized,no_punct,wer
0,yotube,지금까지 얘기한 방 외에 어떤 방이 또 있을까요 보드게임방 이나 방탈출 카페 같은 ...,지금까지 얘기한 방 외에 어떤 방이 또 있을까요 보드게임방 이나 방탈출 카페 같은 ...,0.238806
1,mannual,지금까지 얘기한 방 외에 어떤 방이 또 있을까요? 보드 게임방이나 방탈출 카페 같은...,지금까지 얘기한 방 외에 어떤 방이 또 있을까요 보드 게임방이나 방탈출 카페 같은 ...,0.000000
2,whisper,지금까지 얘기한 방외의 어떤 방이 또 있을까요? 모드게임 방이나 방탈출 카페 같은...,지금까지 얘기한 방외의 어떤 방이 또 있을까요 모드게임 방이나 방탈출 카페 같은 ...,0.149254
3,m4t,지금까지 얘기한 방 외에 어떤 방이 또 있을까요? 보드게임방이나 방탈출 카페 같은 ...,지금까지 얘기한 방 외에 어떤 방이 또 있을까요 보드게임방이나 방탈출 카페 같은 것...,0.179104
4,wav2vec,지금까지 얘기한 방외에 어떤 방이 또 있을까요 모득게인방이나 방달출카에 같은 것도 ...,지금까지 얘기한 방외에 어떤 방이 또 있을까요 모득게인방이나 방달출카에 같은 것도 ...,0.552239


In [24]:
recognized.to_csv('recognized.tsv', sep='\t', index=False, header=True)